# 주의!

train 파일과 test(infer) 파일 **모두**에 아래의 wandb 함수들을 박아두는 게 안전합니다.  
run 이름을 바꿔서 구분하면 되니까 **일단 백업합시다!!**

## wandb.init()

먼저 wandb를 설치합니다.

In [ ]:
!pip install wandb

init() 함수로 프로젝트를 생성합니다.  
'config' parameter로 hyperparameter 값이 정리된 dictionary를 init()에 넘겨주고, 편리한 사용을 위해 이를 로컬 변수 CFG에 저장합니다.   
백업을 위해 프로젝트의 루트 디렉토리를 wandb에게 알려주려면 'settings' parameter에 아래와 같은 형식으로 넘겨줍니다.

In [26]:
import wandb

root_dir = '/content/test-project'
wandb.init(
    project="wandb-pytorch-tutorial",
    name="baseline",
    config={
      "learning_rate": 0.02,
      "model": "resnet18",
      "dataset": "CIFAR-100",
      "epochs": 10,
    },

    # to save all codes in current dir and all subdirs as an artifact
    # https://docs.wandb.ai/guides/app/features/panels/code
    settings=wandb.Settings(code_dir=root_dir),
)

CFG = wandb.config

wandb: WARNING No relevant files were detected in the specified directory. No code will be logged to your run.


In [27]:
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

## wandb.log()

metrics logging이 필요한 위치(각 epoch, 각 step, validation의 가장 아래 등)에 wandb.log()를 사용합니다.  
parameter로 넘겨진 metrics에 대해 wandb에서 자동으로 그래프를 생성해줍니다.  
**반드시 dictionary 형식으로 넘겨줘야 합니다.**

되도록 'step' parameter를 이용하여 그래프의 가로축을 지정하세요. 그렇지 않다면 루프가 여러 번 실행될 때 그래프가 이전 실행에 이어서 계속 그려집니다..  
만약 log를 초기화하고 싶다면 wandb.init()을 재실행해서 새로운 run을 만드세요.

In [28]:
import random

# CFG의 값과 실제 hyperparameter 값을 이런 식으로 반드시 연동해줘야 합니다.
epochs = CFG['epochs']

offset = random.random() / 5
for epoch in range(1, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # accuracy와 loss 값이 wandb에 metric으로서 전달됩니다.
    # 그래프의 가로축에 해당하는 변수를 직접 지정하고 싶다면
    # 'step' parameter를 이용하세요
    wandb.log({"acc": acc, "loss": loss}, step=epoch)

## wandb.save()

실행 과정에서 생성되는 파일을 저장하고 싶다면 **wandb.finish() 직전에** wandb.save()를 실행해주세요.  
저장된 파일들은 해당 run의 **'files'** 항목에서 확인할 수 있습니다.  

형식: wandb.save(저장할 파일명 또는 경로, base_path=루트 디렉토리)

In [29]:
# @title
import os
import pandas as pd
import torch

if not os.path.exists(f'{root_dir}/model'):
    os.makedirs(f'{root_dir}/model')
if not os.path.exists(f'{root_dir}/result'):
    os.makedirs(f'{root_dir}/result')

model = torch.hub.load('pytorch/vision:v0.10.0', CFG['model'], pretrained=True)
submission = pd.read_csv(f'/content/sample_data/mnist_test.csv')

model_path = f'{root_dir}/model/model.pth'
torch.save(model.state_dict(), model_path)

submission_path = f'{root_dir}/result/submission.csv'
submission.to_csv(submission_path)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [30]:
wandb.save(model_path, base_path=root_dir)
wandb.save(submission_path, base_path=root_dir)

['/content/wandb/run-20231125_095129-f7rodjty/files/result/submission.csv']

파일 경로는 엉망이어도 되니 일단 전부 저장해야 한다면...

In [31]:
# set wandb to save all codes, weights, and results
wandb.save("*.py")
wandb.save("*.ipynb")

wandb.save("*.pt")
wandb.save("*.pth")
wandb.save("*.hdf5")

wandb.save("*.csv")

wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.


[]

## wandb.finish()

코드 실행이 끝나기 전 wandb.finish()를 호출해야 wandb에서도 제대로 실행이 마무리된 걸로 처리됩니다.  

In [32]:
wandb.finish()

acc,▁▄▇▇█████
loss,█▄▃▂▁▂▁▁▂
acc,0.88426
loss,0.17696
